# Cosmology
Here are how to use the MRNN to do some cosmological predictions. To specify, the intrinsic luminosity of SNe.  

In [1]:
import os
import glob
import yaml
import keras
import astropy
import numpy as np
import pandas as pd
import keras.backend as K
import astropy.units as u
import astropy.constants as c
from astropy.time import Time
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from keras.models import Sequential,Model
from dust_extinction.averages import GCC09_MWAvg
from sklearn.model_selection import train_test_split
from dust_extinction.parameter_averages import CCM89,F99

Using TensorFlow backend.


## The B-band Magnitude Measure
I got a rough B-band transmission, hope it is accurate enough.  
And when you are using this B-band calculator, please check the wavelength sampling. In our spectra which has 2000 pixels between 2000 and 10000 Angstrom wavelength, the delta frequency is the default number.  

In [2]:
def BFilterMagCalculator(spec,delta_frequency=5.9958492*10**11*u.Hz):
    #shortwave=2000
    #longwave=10000
    spec=pd.DataFrame(spec,columns=['wave','flux'])
    #flux=spec[spec['wave']>shortwave]
    #flux=flux[flux['wave']<longwave]
    wave=np.array(spec['wave'])
    flux=np.array(spec['flux'])
    Bfilter=np.array([0.0,0.0,0.030,0.134,0.567,
                      0.920,0.978,1.000,0.978,
                      0.935,0.853,0.740,0.640,
                      0.536,0.424,0.325,0.235,
                      0.150,0.095,0.043,0.009,0.0,0.0])
    Bwave=np.array([2000,3600,3700,3800,3900,
                    4000,4100,4200,4300,
                    4400,4500,4600,4700,
                    4800,4900,5000,5100,
                    5200,5300,5400,5500,5600,10000])
    Bfunc=interp1d(Bwave,Bfilter)#The Johnson B filter
    flux=flux*Bfunc(wave)
    flux=flux/(1.1964952*10**40)#The spherical luminosity to the flux in a square centimeter at 10 parsec away. 
    delta_lambda=(wave*u.AA)**2*delta_frequency/c.c
    delta_lambda=delta_lambda.to(unit=u.AA)
    flux=sum(flux*delta_lambda.value)
    #print(flux*(1.1964952*10**40))
    #print(flux)
    mag=-2.5*np.log10(flux/6.00069/10**-6)+0.03# Here it is the flux divided by Vega's apparent flux in B band by HST STIS measurement. 
    return mag

In [6]:
def Normalizer(spec,shortwave=6500,longwave=7500):
    small=np.argmin(abs(spec[:,0]-shortwave))
    long=np.argmin(abs(spec[:,0]-longwave))
    if small<long:spec[:,1]=spec[:,1]/np.average(spec[small:long,1])
    if small>long:spec[:,1]=spec[:,1]/np.average(spec[long:small,1])
    return spec

## Read Flux
And also calculate the b band absolute magnitude, then normalize the spectra. 

In [ ]:
X=np.load('DataSet/X.npy')
Y=np.zeros(X.shape[0])
wave=np.load('DataSet/wave.npy')
for i in range(X.shape[0]):
    bmag=BFilterMagCalculator(np.array([wave,X[i].flatten()]).T)
    if bmag>-17.5:continue
    spdata=np.array([wave,X[i]]).T
    spdata=Normalizer(spdata)
    if np.max(spdata[:,1])>30:continue
    Y[i]=bmag
X=X/np.average(X[:,167:269])
magaver=-19
magstd=1
Y=Y-magaver/magstd
Y=np.tanh(Y)/2+0.5
Y=pd.DataFrame(Y,columns=[str(i)+'_'+str(j) for i in range(6,29) for j in range(1,5)])

## Read the data part 2
This part is the original part to read the data into the python program.  

In [ ]:
data=[]
for i in [1,2,3,4,5,6,7,8,9,13,14,15,16,17,18,19,20,21]:
    data.append(pd.read_csv('csvs/'+str(i)+'.csv'))

In [ ]:
specdirs=['Spec/'+str(i)+'/*' for i in [1,2,3,4,5,6,7,8,9,13,14,15,16,17,18,19,20,21]]
specs=[]
speccounts=[]
for i in specdirs:
    specsmall=[]
    speccountsmall=[]
    for j in glob.glob(i):
        #if int(j.split('/')[2].split('.txt')[0])
        k=np.genfromtxt(j)
        specsmall.append(k)
        speccountsmall.append(int(j.split('/')[2].split('.txt')[0]))
    print(i)
    specs.append(specsmall)
    speccounts.append(speccountsmall)

In [ ]:
X=[]
Y=[]
Mags=[]
for i in range(len(specs)):
    for j in range(len(specs[i])):
        spdata=specs[i][j]
        bmag=BFilterMagCalculator(spdata).copy()
        if bmag>-17.5:continue
        spdata=Normalizer(spdata)
        if np.max(spdata[:,1])>30:continue
        Mags.append(bmag)
        X.append(spdata[:,1])
        Y.append(np.array(data[i].iloc[speccounts[i][j],3:]))
X=np.array(X)
X=X.reshape(X.shape[0],2000,1)
Y=np.array(Y)
Y=pd.DataFrame(Y,columns=['mag'])
Y=(Y-Yaverage)/Ystd
Y=np.tanh(Y)/2+0.5
Y=pd.DataFrame(Y,columns=[str(i)+'_'+str(j) for i in range(6,29) for j in range(1,5)])
Mags=np.array(Mags)

magaver=-19
magstd=1

Ymag=np.tanh((Mags-magaver)/magstd)/2+0.5
Ymag=pd.DataFrame({'mag':Ymag})

In [8]:
def DataAugmenter(Xoriginal,Yoriginal):
    noisparam1=50
    noisparam2=5
    X2=Xoriginal
    Y2=Yoriginal
    X3=Xoriginal
    F5500=X3[:,400:420].mean(axis=1)
    F5500=F5500.reshape([X3.shape[0],1,1])
    Slist=np.random.random(X3.shape[0])*noisparam1+noisparam2# you can modify the 50 and 5 here depending how much noise you want to add
    Slist=Slist.reshape([X3.shape[0],1,1])
    Noiselist=np.random.randn(X3.shape[0]*X3.shape[1]).reshape([X3.shape[0],X3.shape[1],1])*X3**0.5
    X3=X3*(1+Noiselist/F5500**0.5/Slist)
    Y3=Yoriginal
    for i in range(len(X2)):
        X2[i]=savgol_filter(X2[i].T,\
                                  np.random.choice([7,9,11,13,15,17,19,21,23,25]),\
                                  np.random.choice([2,3,4,5,6])).T
    Xaugment=np.concatenate([Xoriginal,X2,X3])
    Yaugment=pd.concat([Yoriginal,Y2,Y3])
    return Xaugment,Yaugment

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,train_size=0.9)
X_train,Y_train=DataAugmenter(X_train,Y_train)
X_train,Y_train=DataAugmenter(X_train,Y_train)

X_test2,Y_test2=DataAugmenter(X_test,Y_test)
X_test3,Y_test3=DataAugmenter(X_test2,Y_test2)

In [ ]:
def UltraDenseResNeuro(NameShell='26_3',CellNumber=7,\
                       X_train=X_train,X_test=X_test,Y_train=Y_train,Y_test=Y_test,usegpu=True):
    INput=Input(shape=(X_train.shape[1],1,))
    conv1=Conv1D(8,15,strides=2,padding='same')(INput)
    conv1=Conv1D(16,3,strides=1,padding='same')(conv1)
    batc1=BatchNormalization()(conv1)
    acti1=Activation('relu')(batc1)
    pool1=MaxPooling1D(2)(acti1)
    
    conv2=Conv1D(8,1)(pool1)
    batc2=BatchNormalization()(conv2)
    acti2=Activation('relu')(batc2)
    conv3=Conv1D(16,3,padding='same')(acti2)
    
    add1=Add()([pool1,conv3])
    
    conv2=Conv1D(8,1)(add1)
    batc2=BatchNormalization()(conv2)
    acti2=Activation('relu')(batc2)
    conv3=Conv1D(16,3,padding='same')(acti2)
    
    adds=[add1]
    addi=Add()(adds+[conv3])
    adds.append(addi)
    
    for i in range(CellNumber):
        conv2=Conv1D(8,1)(addi)
        batc2=BatchNormalization()(conv2)
        acti2=Activation('relu')(batc2)
        conv3=Conv1D(16,3,padding='same')(acti2)
        addi=Add()(adds+[conv3])
        adds.append(addi)
    
    batc2=BatchNormalization()(addi)
    flat1=keras.layers.Flatten()(batc2)
    drop1=Dropout(0.2)(flat1)
    dens1=Dense(256,activation='relu')(drop1)
    drop2=Dropout(0.2)(dens1)
    dens2=Dense(128,activation='relu')(drop2)
    dens3=Dense(1,activation='sigmoid')(dens2)
    
    model=Model(inputs=INput,outputs=dens3)
    print(model.summary())
    if usegpu==True:model=keras.utils.multi_gpu_model(model,gpus=2)
    opt=keras.optimizers.adam(lr=0.000003,decay=1e-6)
    model.compile(optimizer=opt,loss='mse')    
    history1=model.fit(X_train,Y_train[NameShell],epochs=700,\
              validation_data=[X_test,Y_test[NameShell]],batch_size=4000,\
              callbacks=[keras.callbacks.EarlyStopping(patience=1)],verbose=1)
    opt=keras.optimizers.adam(lr=0.00000003,decay=1e-6)
    model.compile(optimizer=opt,loss='mse')
    history2=model.fit(X_train,Y_train[NameShell],epochs=700,\
              validation_data=[X_test,Y_test[NameShell]],batch_size=10000,\
              callbacks=[keras.callbacks.EarlyStopping(patience=1)],verbose=1)
    return model,history1,history2

## Run it many times
Because this neural network is not that stable on the luminosity dataset, I tried to run the prediction model several times, and store the best one. 

In [ ]:
model,his1,his2=UltraDenseResNeuro(NameShell='mag',usegpu=True)
memory=his2.history['val_loss'][0]
while True:
    model,his1,his2=UltraDenseResNeuro(NameShell='mag',usegpu=True)
    if his2.history['val_loss'][0]<memory:
        memory=his2.history['val_loss'][0]
        model.save('MdSaver/Cosmo/Naive'+str(round(memory,9))+'.hdf')
        Yout=model.predict(X_test,batch_size=2000)
        np.save('DataCache/Cosmo/YoutNaive'+str(round(memory,9))+'.npy',Yout)
        np.save('DataCache/Cosmo/XtestNaive.npy',X_test)
        Y_test.to_csv('DataCache/Cosmo/YtestNaive.csv')

In [ ]:
def BMagPred(spectra,z):
    mag=np.arctanh(2*(Predictor(model,spectra,z)[0][0]-0.5))*magstd+magaver
    return mag

In [ ]:
spectralist=[
    'ObserveSpectra/Prediction/SN2011fe/SN2011fe_0.4d.dat',
    'ObserveSpectra/Prediction/SN2011fe/SN2011fe_-2.6d.dat',
    'ObserveSpectra/Prediction/SN2011fe/SN2011fe_3.7d.dat',
    'ObserveSpectra/Prediction/SN2013dy/SN2013dy_-3.1d.flm',
    'ObserveSpectra/Prediction/SN2013dy/SN2013dy_-1.1d.flm',
    'ObserveSpectra/Prediction/SN2013dy/SN2013dy_0.9d.flm',
    'ObserveSpectra/Prediction/SN2013dy/SN2013dy_3.9d.flm',
    'ObserveSpectra/Prediction/SN2011iv/SN2011iv_0.6d.flm',
    'ObserveSpectra/Prediction/SN2015F/SN2015F_-2.3d.flm',
    'ObserveSpectra/Prediction/ASASSN-14lp/ASASSN-14lp_-4.4d.flm',
    'ObserveSpectra/Prediction/SN2011by/SN2011by_-0.4d.flm']
ext1list=[CCM89(Rv=3.1),CCM89(Rv=3.1),CCM89(Rv=3.1),
         CCM89(Rv=3.1),CCM89(Rv=3.1),CCM89(Rv=3.1),CCM89(Rv=3.1),
         CCM89(Rv=3.1),CCM89(Rv=3.1),CCM89(Rv=3.1),F99(Rv=3.1)]
ext2list=[GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg(),
         GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg(),
         GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg(),GCC09_MWAvg()]
ext1EbvList=[0,0,0,0.206,0.206,0.206,0.206,0,0.035,0.33,0.039]
ext2EbvList=[0,0,0,0.135,0.135,0.135,0.135,0,0.175,0.021,0.013]
zlist=[0.000804,0.000804,0.000804,0.00389,0.00389,0.00389,0.00389,0.006494,0.0049,0.0051,0.002843]
snname=['SN2011fe_0.4d','SN2011fe_-2.6d','SN2011fe_3.7d','SN2013dy_-3.1d','SN2013dy_-1.1d',
        'SN2013dy_0.9d','SN2013dy_3.9d','SN2011iv_0.6d','SN2015F_-2.3d','ASASSN-14lp_-4.4d','SN2011by_-0.4d']

## The Prediction
I write the prediction functions here, the best model I have and its relating testing dataset are also shown below. 

In [ ]:
Yout=np.load('DataCache/Cosmo/Yout2018.3.10.000432561.npy')
Yreal=pd.read_csv('DataCache/Cosmo/Ytest2018.3.1.csv')
model=keras.models.load_model('MdSaver/Cosmo/Aver-19_Std1_NBF_2018.3.10.000432561.hdf')

In [ ]:
ObserveYoutData=[]
for i in range(11):
    spr=np.genfromtxt(spectralist[i])
    ObserveYoutData.append(NewPredictor(spr,ext1list[i],ext2list[i],ext1EbvList[i],ext2EbvList[i],zlist[i]))
ObserveYoutData=np.array(ObserveYoutData).flatten()
for i in range(len(ObserveYoutData)):
    predmag=np.arctanh(2*(ObserveYoutData[i]-0.5))*magstd+magaver    
    lower,upper=OneSigmaCalculator(Yout,Yreal['mag'],center=ObserveYoutData[i])
    middle=MiddleCalculator(Yout,Yreal['mag'],center=ObserveYoutData[i])
    lowmag=np.arctanh(2*(lower-0.5))*magstd+magaver
    uppmag=np.arctanh(2*(upper-0.5))*magstd+magaver
    midmag=np.arctanh(2*(middle-0.5))*magstd+magaver
    print(snname[i],lowmag,midmag,predmag,uppmag)
    print(lowmag-midmag,uppmag-midmag)